In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from kneed import KneeLocator
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from scipy.cluster.hierarchy import fcluster
from scipy.spatial.distance import pdist, squareform
year=2015
def save_cluster_data(year):
    df=pd.read_csv(f'cluster/cluster_{year}.csv')
    #testdf = df[(df.MPG > 23) & (df.GP > 15) & (df.SEASON == '2022-23')].reset_index(drop=True)
    
    season=str(year-1)+'-'+str(year)[-2:]
    print(season)
    testdf = df[(df.MPG > 10) & (df.GP > 10) & (df.SEASON == season)].reset_index(drop=True)
    print(testdf)
    
    features = [x for x in df.columns if (x != 'PLAYER_NAME') &  (x != 'POSITION') & (x != 'SEASON')]
    
    x = testdf.loc[:, features].values
    y = testdf.loc[:,['PLAYER_NAME']].values
    
    x = StandardScaler().fit_transform(x) # standardize all values
    
    pca = PCA(n_components=0.99)
    principalComponents = pca.fit_transform(x)
    
    '''
    plt.figure(figsize=(10,65))
    plt.title('2024-25 NBA Hierarchical Clustering Dendrogram')
    dend = shc.dendrogram(shc.linkage(x, method='ward'), labels=list(testdf.PLAYER_NAME), orientation='left')
    
    plt.yticks(fontsize=8)
    plt.xlabel('Height')
    
    plt.tight_layout()
    '''
    # Save the plot in PNG format
    #plt.savefig("nba_hierarchical_clustering_dendrogram.png", format='png', dpi=300)
    
    # Optionally, display the plot as well
    #plt.show()
    
    
    # In[3]:
    
    
    
    
    # Assuming 'principalComponents' contains the PCA-transformed features and 'testdf' contains the player names
    # Here 'x' is 'principalComponents' and 'y' is 'testdf['PLAYER_NAME']'
    
    # Convert the PCA feature vectors into a square-form distance matrix
    x = principalComponents  # PCA-transformed feature matrix
    y = testdf['PLAYER_NAME'].values  # Player names
    z = testdf['PLAYER_ID'].values  # Player IDs
    
    # Apply hierarchical clustering 
    cluster_labels = fcluster(shc.linkage(x, method='ward'), t=25, criterion='distance')
    
    # Create a DataFrame mapping players to their cluster labels and IDs
    player_cluster_mapping = pd.DataFrame({
        'PLAYER_NAME': y, 
        'PLAYER_ID': z,
        'CLUSTER': cluster_labels
    })
    
    x = principalComponents  # PCA-transformed feature matrix
    y = testdf['PLAYER_NAME'].values  # Player names
    z = testdf['PLAYER_ID'].values  # Player IDs
    
    # Apply hierarchical clustering 
    cluster_labels = fcluster(shc.linkage(x, method='ward'), t=25, criterion='distance')
    
    # Create a DataFrame mapping players to their cluster labels and IDs
    player_cluster_mapping = pd.DataFrame({
       'PLAYER_NAME': y, 
       'PLAYER_ID': z,
       'CLUSTER': cluster_labels
    })
    
    # Prepare a DataFrame to hold each player and their 5 most similar players
    similar_players_df = pd.DataFrame()
    for idx, player in enumerate(y):
       # Get the current player's cluster and ID
       player_row = player_cluster_mapping[player_cluster_mapping['PLAYER_NAME'] == player]
       cluster = player_row['CLUSTER'].iloc[0]
       base_player_id = player_row['PLAYER_ID'].iloc[0]
       
       # Get indices of players in the same cluster
       indices = player_cluster_mapping[player_cluster_mapping['CLUSTER'] == cluster].index
       
       # Calculate distances from the current player to others in the same cluster
       distances = euclidean_distances(x[indices], [x[idx]]).flatten()
       
       # Get indices of the 5 closest players
       closest_indices = np.argsort(distances)[1:6]  # Exclude the closest (itself)
       
       # Extract names, IDs, and distances of the 5 closest players
       closest_players = player_cluster_mapping.iloc[indices[closest_indices]]
       closest_distances = distances[closest_indices]
       
       # Create a new row with player name and ID, similar player names, IDs, and distances
       new_row = pd.DataFrame([{
           'PLAYER_NAME': player,
           'PLAYER_ID': base_player_id,
           'SIMILAR_1_NAME': closest_players['PLAYER_NAME'].iloc[0] if len(closest_players) > 0 else '',
           'SIMILAR_1_ID': closest_players['PLAYER_ID'].iloc[0] if len(closest_players) > 0 else '',
           'SIMILAR_1_DISTANCE': closest_distances[0] if len(closest_distances) > 0 else '',
           'SIMILAR_2_NAME': closest_players['PLAYER_NAME'].iloc[1] if len(closest_players) > 1 else '',
           'SIMILAR_2_ID': closest_players['PLAYER_ID'].iloc[1] if len(closest_players) > 1 else '',
           'SIMILAR_2_DISTANCE': closest_distances[1] if len(closest_distances) > 1 else '',
           'SIMILAR_3_NAME': closest_players['PLAYER_NAME'].iloc[2] if len(closest_players) > 2 else '',
           'SIMILAR_3_ID': closest_players['PLAYER_ID'].iloc[2] if len(closest_players) > 2 else '',
           'SIMILAR_3_DISTANCE': closest_distances[2] if len(closest_distances) > 2 else '',
           'SIMILAR_4_NAME': closest_players['PLAYER_NAME'].iloc[3] if len(closest_players) > 3 else '',
           'SIMILAR_4_ID': closest_players['PLAYER_ID'].iloc[3] if len(closest_players) > 3 else '',
           'SIMILAR_4_DISTANCE': closest_distances[3] if len(closest_distances) > 3 else '',
           'SIMILAR_5_NAME': closest_players['PLAYER_NAME'].iloc[4] if len(closest_players) > 4 else '',
           'SIMILAR_5_ID': closest_players['PLAYER_ID'].iloc[4] if len(closest_players) > 4 else '',
           'SIMILAR_5_DISTANCE': closest_distances[4] if len(closest_distances) > 4 else ''
       }])
       
       similar_players_df = pd.concat([similar_players_df, new_row], ignore_index=True)
    
    # Export the DataFrame to a CSV file
    similar_players_df.to_csv(f'{year}_similar_players.csv', index=False)
    print("CSV file 'nba_similar_players.csv' has been created with each player and their 5 most similar players.")
    
    
    # In[4]:
    
    # Filter the DataFrame
    testdf = df[(df.MPG > 12) & (df.GP > 10) & (df.SEASON == season)].reset_index(drop=True)
    
    # Select features for clustering
    features = [x for x in df.columns if (x != 'PLAYER_NAME') & (x != 'POSITION') & (x != 'SEASON')]
    
    # Standardize the data
    x = testdf.loc[:, features].values
    x = StandardScaler().fit_transform(x)
    
    # Perform PCA to reduce dimensionality
    pca = PCA(n_components=2)  # Reduce to 2 components for visualization
    principalComponents = pca.fit_transform(x)
    testdf['PCA1'] = principalComponents[:, 0]
    testdf['PCA2'] = principalComponents[:, 1]
    
    
        
        # Add PCA components to dataframe
    for i in range(principalComponents.shape[1]):
        testdf[f'PCA_{i+1}'] = principalComponents[:, i]
    print(testdf.columns)
    # Determine optimal clusters using elbow method and silhouette scores
    inertia = []
    silhouette_scores = []
    k_range = range(2, 15)  # Range of cluster numbers to test
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(principalComponents)
        inertia.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(principalComponents, kmeans.labels_))
    
    # Find the elbow point using KneeLocator
    knee_locator = KneeLocator(k_range, inertia, curve="convex", direction="decreasing")
    optimal_clusters_elbow = knee_locator.knee
    
    # Find optimal clusters by silhouette score
    optimal_clusters_silhouette = k_range[np.argmax(silhouette_scores)]
    
    # Plot the elbow method
    '''
    plt.figure(figsize=(8, 5))
    plt.plot(k_range, inertia, 'bo-', label='Inertia')
    plt.axvline(optimal_clusters_elbow, color='r', linestyle='--', label=f'Elbow Point: {optimal_clusters_elbow}')
    plt.title("Elbow Method")
    plt.xlabel("Number of Clusters")
    plt.ylabel("Inertia")
    plt.legend()
    plt.tight_layout()
    plt.savefig("nba_pca_kmeans_elbow.png", format='png', dpi=300)
    #plt.show()
    
    # Plot silhouette scores
    plt.figure(figsize=(8, 5))
    plt.plot(k_range, silhouette_scores, 'go-', label='Silhouette Score')
    plt.axvline(optimal_clusters_silhouette, color='r', linestyle='--', label=f'Best Silhouette: {optimal_clusters_silhouette}')
    plt.title("Silhouette Scores")
    plt.xlabel("Number of Clusters")
    plt.ylabel("Silhouette Score")
    plt.legend()
    plt.tight_layout()
    plt.savefig("nba_pca_kmeans_silhouette.png", format='png', dpi=300)
    #plt.show()
    '''
    # Let user choose optimal clusters
    print(f"Optimal clusters (Elbow): {optimal_clusters_elbow}")
    print(f"Optimal clusters (Silhouette): {optimal_clusters_silhouette}")
    try:
        #user_clusters = int(input("Enter the number of clusters you prefer (press Enter to use the best silhouette score): ").strip() or optimal_clusters_silhouette)
        user_clusters=8
    except ValueError:
        user_clusters = optimal_clusters_silhouette  # Fallback to silhouette-based optimal clusters
    
    print(f"Using {user_clusters} clusters for final clustering.")
    
    # Perform KMeans clustering with the chosen number of clusters
    kmeans_final = KMeans(n_clusters=user_clusters, random_state=42)
    kmeans_final.fit(principalComponents)
    testdf['Cluster'] = kmeans_final.labels_
    
    testdf.to_csv(f'cluster/nba_analysis_{year}.csv',index=False)
    






for year in range(2014,2026):
    save_cluster_data(year)


2013-14
     PLAYER_ID      PLAYER_NAME   AGE  GP          MIN       FGM        FGA  \
0       201166     Aaron Brooks  29.0  72  1557.271667  5.386343  13.431183   
1         1733    Al Harrington  34.0  34   511.365000  5.772785  14.572761   
2       201143       Al Horford  28.0  29   957.730000  8.946154  15.787330   
3         2744     Al Jefferson  29.0  73  2553.238333  9.869819  19.401244   
4       202329  Al-Farouq Aminu  23.0  80  2045.426667  4.118456   8.694519   
..         ...              ...   ...  ..          ...       ...        ...   
367       2584     Willie Green  32.0  55   869.250000  4.224331  11.223469   
368     201163  Wilson Chandler  27.0  62  1926.720000  5.736173  13.789238   
369     202333     Xavier Henry  23.0  43   908.176667  5.708140  13.675753   
370       2216    Zach Randolph  32.0  79  2705.005000  7.452851  15.943778   
371       2585    Zaza Pachulia  30.0  53  1324.575000  4.049601   9.485307   

         FG3M      FG3A       FTM  ...  PCT

/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabrie

Optimal clusters (Elbow): 5
Optimal clusters (Silhouette): 2
Using 8 clusters for final clustering.


/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


2014-15
     PLAYER_ID      PLAYER_NAME   AGE  GP          MIN       FGM        FGA  \
0       201985         AJ Price  28.0  26   323.490000  5.675600  15.246221   
1       201166     Aaron Brooks  30.0  82  1884.613333  6.571109  15.606384   
2       203932     Aaron Gordon  19.0  47   796.925000  4.201148   9.396116   
3       203940    Adreian Payne  24.0  32   739.638333  4.429192  10.707936   
4       201143       Al Horford  29.0  76  2318.166667  8.059817  14.985980   
..         ...              ...   ...  ..          ...       ...        ...   
393       2584     Willie Green  33.0  52   951.196667  4.465953  11.581201   
394     201163  Wilson Chandler  28.0  78  2471.151667  6.089468  14.203904   
395     203897      Zach LaVine  20.0  77  1901.721667  5.414041  12.815756   
396       2216    Zach Randolph  33.0  71  2303.590000  7.095013  14.565092   
397       2585    Zaza Pachulia  31.0  73  1729.666667  4.995182  11.010214   

         FG3M      FG3A       FTM  ...  PCT

...

/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabrie

Optimal clusters (Elbow): 5
Optimal clusters (Silhouette): 2
Using 8 clusters for final clustering.


/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


2017-18
     PLAYER_ID      PLAYER_NAME   AGE  GP          MIN       FGM        FGA  \
0       203932     Aaron Gordon  22.0  58  1909.078333  7.071475  16.311536   
1      1627846      Abdel Nader  24.0  48   521.526667  3.451405  10.285188   
2       201143       Al Horford  32.0  72  2277.025000  5.818118  11.905008   
3         2744     Al Jefferson  33.0  36   483.560000  8.263711  15.485152   
4       202329  Al-Farouq Aminu  27.0  69  2072.171667  3.995808  10.111131   
..         ...              ...   ...  ..          ...       ...        ...   
401    1627812     Yogi Ferrell  25.0  82  2281.485000  4.907330  11.518813   
402    1628380     Zach Collins  20.0  66  1045.450000  3.960017   9.951695   
403     203897      Zach LaVine  23.0  24   656.286667  7.460155  19.473198   
404       2216    Zach Randolph  36.0  59  1507.611667  8.620257  18.219546   
405       2585    Zaza Pachulia  34.0  69   971.746667  5.519957   9.780327   

         FG3M      FG3A       FTM  ...  PCT

/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/gabriel1200/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:



print(testdf)
# Interactive visualization with Plotly
fig = px.scatter(
    testdf,
    x='PCA1',
    y='PCA2',
    color='Cluster',
    hover_data=['PLAYER_NAME'],
    title="NBA Player Clusters (PCA Visualization)",
    template='plotly_white'
)

fig.update_traces(marker=dict(size=10, opacity=0.8))
fig.update_layout(legend_title_text='Cluster')

# Add cluster filtering via selection
fig.update_layout(clickmode='event+select')

# Show the interactive plot
fig.show()

# Save the resulting DataFrame to a CSV
testdf.to_csv("nba_player_clusters.csv", index=False)


# In[28]:


import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances

# Assuming testdf contains PCA-transformed data and original features
# principalComponents is the PCA-transformed array
# features is the list of original feature names

def get_top_features(pca, n_features):
    """
    Get the top n_features most important features based on PCA loadings.
    """
    loading_scores = np.abs(pca.components_).sum(axis=0)
    feature_importance = pd.DataFrame({'Feature': features, 'Importance': loading_scores})
    return feature_importance.sort_values(by='Importance', ascending=False).head(n_features)

# Get top features from PCA
n_top_features = 70
# Adjust as needed
top_features = get_top_features(pca, n_top_features)['Feature'].tolist()
print(top_features)
# Normalize stats for comparison
for col in features:
    testdf[f'{col}_Z'] = (testdf[col] - testdf[col].mean()) / testdf[col].std()



from rapidfuzz import process

def get_cluster_defining_features(cluster_averages, league_averages, n_features=5):
    """
    Identify features that most define the cluster compared to the league.
    """
    differences = (cluster_averages - league_averages).abs()
    defining_features = differences.nlargest(n_features).index.tolist()
    return defining_features

def get_player_unique_features(player_stats, cluster_averages, n_features=5):
    """
    Identify features that most make the player unique within their cluster.
    """
    # Ensure numeric data and handle NaNs
    player_stats = pd.to_numeric(player_stats, errors='coerce')
    cluster_averages = pd.to_numeric(cluster_averages, errors='coerce')

    # Calculate differences and drop NaNs
    differences = (player_stats - cluster_averages).abs().dropna()

    # Select top n features
    unique_features = differences.nlargest(n_features).index.tolist()
    return unique_features




# In[37]:


def search_player_with_weighted_euclidean(player_name, num_similar=5, n_features=5, similarity_threshold=80):
    """
    Enhanced search function using Weighted Euclidean Distance for similarity calculation.

    Args:
        player_name (str): Name of the player to search for.
        num_similar (int): Number of similar players to find.
        n_features (int): Number of features for cluster and uniqueness analysis.
        similarity_threshold (int): Minimum similarity threshold for fuzzy matching.

    Returns:
        tuple: Similar players and feature comparisons.
    """
    def fuzzy_match(name):
        if name not in testdf['PLAYER_NAME'].values:
            player_names = testdf['PLAYER_NAME'].values
            matches = process.extract(name, player_names, limit=3)
            close_matches = [match[0] for match in matches if match[1] >= similarity_threshold]
            if close_matches:
                print(f"Player '{name}' not found. Did you mean:")
                for i, match in enumerate(close_matches, 1):
                    print(f"{i}. {match}")
                try:
                    choice = int(input(f"Enter the number of the correct player for '{name}' (or 0 to cancel): ").strip())
                    if choice == 0:
                        print("Operation cancelled.")
                        return None
                    elif 1 <= choice <= len(close_matches):
                        return close_matches[choice - 1]
                    else:
                        print("Invalid choice. Please try again.")
                        return None
                except ValueError:
                    print("Invalid input. Please enter a number.")
                    return None
            else:
                print(f"No close matches found for '{name}'. Please try again.")
                return None
        return name

    # Fuzzy match the player name
    player_name = fuzzy_match(player_name)
    if player_name is None:
        return None, None

    # Get player data
    player_data = testdf[testdf['PLAYER_NAME'] == player_name]
    player_pca = player_data[['PCA1', 'PCA2']].values
    player_cluster = player_data['Cluster'].iloc[0]

    cluster_data = testdf[testdf['Cluster'] == player_cluster]
    cluster_pca = cluster_data[['PCA1', 'PCA2']].values

    # Compute relative differences for Weighted Euclidean Distance
    relative_differences = np.array([
        [
            abs(player_pca[0][i] - point[i]) / ((player_pca[0][i] + point[i]) / 2) if (player_pca[0][i] + point[i]) != 0 else 0
            for i in range(cluster_pca.shape[1])
        ]
        for point in cluster_pca
    ])

    # Compute Weighted Euclidean Distance
    distances = np.sqrt(np.sum(relative_differences**2, axis=1))

    cluster_data = cluster_data.assign(Distance=distances).sort_values(by='Distance')
    similar_players = cluster_data.head(num_similar + 1)
    similar_players = similar_players[similar_players['PLAYER_NAME'] != player_name]

    cluster_averages = testdf.groupby('Cluster')[top_features].mean().loc[player_cluster]
    league_averages = testdf[top_features].mean()
    player_stats = player_data[top_features].iloc[0]

    # Get defining features
    cluster_defining_features = get_cluster_defining_features(cluster_averages, league_averages, n_features)
    player_unique_features = get_player_unique_features(player_stats, cluster_averages, n_features)

    print(f"Most similar players to {player_name} (based on Weighted Euclidean Distance):")
    print(similar_players[['PLAYER_NAME', 'Distance']])

    print("\nCluster-Defining Features:")
    print(pd.DataFrame({
        'Feature': cluster_defining_features,
        'Difference': (cluster_averages[cluster_defining_features] - league_averages[cluster_defining_features]).values
    }))

    print("\nPlayer-Unique Features:")
    print(pd.DataFrame({
        'Feature': player_unique_features,
        'Difference': (player_stats[player_unique_features] - cluster_averages[player_unique_features]).values
    }))

    return similar_players, {
        'Cluster-Defining Features': cluster_defining_features,
        'Player-Unique Features': player_unique_features,
    }

# Example usage
player_name = input("Enter the player name: ").strip()
similar_players, comparison_df = search_player_with_weighted_euclidean(player_name, num_similar=15, n_features=n_top_features)


# In[ ]:





# In[38]:


def compare_two_players_with_relative_diff(player1_name, player2_name, n_features=5, similarity_threshold=80):
    """
    Compare two players and summarize their key similarities and differences, prioritizing relative differences.

    Args:
        player1_name (str): The name of the first player.
        player2_name (str): The name of the second player.
        n_features (int): The number of top features to summarize.
        similarity_threshold (int): The minimum similarity score for fuzzy matching.

    Returns:
        dict: A summary of similarities and differences between the two players, including an overall similarity score.
    """
    def fuzzy_match(name):
        if name not in testdf['PLAYER_NAME'].values:
            player_names = testdf['PLAYER_NAME'].values
            matches = process.extract(name, player_names, limit=3)
            close_matches = [match[0] for match in matches if match[1] >= similarity_threshold]
            if close_matches:
                print(f"Player '{name}' not found. Did you mean:")
                for i, match in enumerate(close_matches, 1):
                    print(f"{i}. {match}")
                try:
                    choice = int(input(f"Enter the number of the correct player for '{name}' (or 0 to cancel): ").strip())
                    if choice == 0:
                        print("Operation cancelled.")
                        return None
                    elif 1 <= choice <= len(close_matches):
                        return close_matches[choice - 1]
                    else:
                        print("Invalid choice. Please try again.")
                        return None
                except ValueError:
                    print("Invalid input. Please enter a number.")
                    return None
            else:
                print(f"No close matches found for '{name}'. Please try again.")
                return None
        return name

    # Fuzzy match for both player names
    player1_name = fuzzy_match(player1_name)
    if player1_name is None:
        return None

    player2_name = fuzzy_match(player2_name)
    if player2_name is None:
        return None

    # Get player data
    player1_data = testdf[testdf['PLAYER_NAME'] == player1_name][top_features].iloc[0]
    player2_data = testdf[testdf['PLAYER_NAME'] == player2_name][top_features].iloc[0]

    # Separate numeric and non-numeric features
    numeric_features = [f for f in top_features if pd.api.types.is_numeric_dtype(testdf[f])]
    non_numeric_features = [f for f in top_features if not pd.api.types.is_numeric_dtype(testdf[f])]

    # Calculate differences for numeric features
    player1_numeric = player1_data[numeric_features].astype(float)
    player2_numeric = player2_data[numeric_features].astype(float)
    differences = (player1_numeric - player2_numeric).abs()
    relative_differences = (differences / ((player1_numeric + player2_numeric) / 2)) * 100

    # Calculate overall similarity score (lower is more similar)
    overall_similarity_score = relative_differences.mean()

    # Find most similar and different features for numeric features
    similar_numeric_features = relative_differences.nsmallest(n_features).index.tolist()
    different_numeric_features = relative_differences.nlargest(n_features).index.tolist()

    # Create summary for non-numeric features
    non_numeric_summary = {
        feature: {
            player1_name: player1_data[feature],
            player2_name: player2_data[feature],
        }
        for feature in non_numeric_features
    }

    # Create final summary
    summary = {
        "Overall Similarity Score (lower is more similar)": round(overall_similarity_score, 2),
        "Numeric Similar Features": {
            feature: {
                player1_name: round(player1_numeric[feature], 2),
                player2_name: round(player2_numeric[feature], 2),
                "Difference": round(differences[feature], 2),
                "Relative Difference (%)": round(relative_differences[feature], 2),
            }
            for feature in similar_numeric_features
        },
        "Numeric Different Features": {
            feature: {
                player1_name: round(player1_numeric[feature], 2),
                player2_name: round(player2_numeric[feature], 2),
                "Difference": round(differences[feature], 2),
                "Relative Difference (%)": round(relative_differences[feature], 2),
            }
            for feature in different_numeric_features
        },
        "Non-Numeric Features": non_numeric_summary
    }

    # Print summary
    print(f"\nComparison between {player1_name} and {player2_name}:\n")
    print(f"Overall Similarity Score (lower is more similar): {summary['Overall Similarity Score (lower is more similar)']}")

    print("\nNumeric Similar Features (lowest relative differences):")
    for feature, stats in summary["Numeric Similar Features"].items():
        print(f"  - {feature}: {player1_name} = {stats[player1_name]}, {player2_name} = {stats[player2_name]}, "
              f"Difference = {stats['Difference']}, Relative Difference = {stats['Relative Difference (%)']}%")

    print("\nNumeric Different Features (highest relative differences):")
    for feature, stats in summary["Numeric Different Features"].items():
        print(f"  - {feature}: {player1_name} = {stats[player1_name]}, {player2_name} = {stats[player2_name]}, "
              f"Difference = {stats['Difference']}, Relative Difference = {stats['Relative Difference (%)']}%")

    print("\nNon-Numeric Features:")
    for feature, stats in summary["Non-Numeric Features"].items():
        print(f"  - {feature}: {player1_name} = {stats[player1_name]}, {player2_name} = {stats[player2_name]}")

    return summary

# Example usage
player1 = input("Enter the first player's name: ").strip()
player2 = input("Enter the second player's name: ").strip()
comparison_summary = compare_two_players_with_relative_diff(player1, player2, n_features=20)


# In[ ]:





# In[ ]:





# In[9]:


def find_most_similar_and_different_players(testdf, top_features):
    """
    Find the most similar and most different players based on Weighted Euclidean Distance.

    Args:
        testdf (pd.DataFrame): DataFrame containing player data.
        top_features (list): List of features to use for comparison.

    Returns:
        dict: A dictionary containing the most similar and most different pairs of players.
    """
    from itertools import combinations

    # Extract player names and relevant feature data
    player_names = testdf["PLAYER_NAME"].values
    player_data = np.nan_to_num(
        testdf[top_features].values, nan=0.0, posinf=0.0, neginf=0.0
    )

    # Initialize variables to store results
    most_similar_pair = None
    most_different_pair = None
    min_similarity_score = float("inf")  # Smallest distance = most similar
    max_similarity_score = -float("inf")  # Largest distance = most different

    # Iterate over all unique pairs of players
    for player1, player2 in combinations(range(len(player_names)), 2):
        # Extract feature data for the two players
        player1_data = player_data[player1]
        player2_data = player_data[player2]

        # Calculate relative differences safely
        differences = np.abs(player1_data - player2_data)
        denominator = (player1_data + player2_data) / 2

        # Replace zero denominators with a small epsilon value
        denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)

        # Calculate relative differences
        relative_differences = (differences / denominator) * 100

        # Calculate Weighted Euclidean Distance
        similarity_score = np.sqrt(np.sum(relative_differences**2))

        # Update most similar pair
        if similarity_score < min_similarity_score:
            min_similarity_score = similarity_score
            most_similar_pair = (player_names[player1], player_names[player2])

        # Update most different pair
        if similarity_score > max_similarity_score:
            max_similarity_score = similarity_score
            most_different_pair = (player_names[player1], player_names[player2])

    # Return results
    return {
        "Most Similar Pair": {
            "Players": most_similar_pair,
            "Similarity Score": round(min_similarity_score, 2),
        },
        "Most Different Pair": {
            "Players": most_different_pair,
            "Similarity Score": round(max_similarity_score, 2),
        },
    }


# Example usage
results = find_most_similar_and_different_players(testdf, top_features)
print("Most Similar Pair:", results["Most Similar Pair"])
print("Most Different Pair:", results["Most Different Pair"])


# In[ ]:
